In [32]:
import sys # for automation and parallelization: set manual to false when run by a launcher
import json
 
default = {'scenario': 'port-moresby', 'training_folder':'../..'} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))


In [33]:
import os
import time
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)
from syspy.spatial import spatial, utils
io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

num cores: 8


In [34]:
scenario = argv['scenario']

on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)

training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'

if not on_lambda:
    scenario_folder = training_folder + '/scenarios/' + scenario  + '/inputs/'
    output_folder = training_folder + '/scenarios/' + scenario + '/outputs/'
    model_folder = training_folder + '/scenarios/' + scenario  + '/model/'
else:
    scenario_folder = input_folder
    output_folder = training_folder + '/outputs/'
    model_folder = training_folder + '/model/'
print('input folder: ', input_folder)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folder)
print('model folder : ', model_folder)

On Lambda :  False
input folder:  ../../inputs/
output folder:  ../../scenarios/port-moresby/outputs/
scen folder :  ../../scenarios/port-moresby/inputs/
model folder :  ../../scenarios/port-moresby/model/


In [35]:
if 'params' in argv.keys():
    scenario = argv['params']['general']['scenario'] #'params':{'general':{'scenario':'ref2023'}}
    var = excel.read_var(file=input_folder+'parameters.xlsx', scenario=scenario, return_ancestry=False)
    var.update(pd.DataFrame.from_dict(argv['params'], orient="index").stack())
else:
    var = excel.read_var(file=input_folder+'parameters.xlsx', scenario=scenario, return_ancestry=False)

In [36]:
def round_up(n, decimals=0):
    multiplier = 10**decimals
    return math.ceil(n * multiplier) / multiplier

# inputs

In [37]:
sm = stepmodel.read_zippedpickles(model_folder +'logit_assignment')
sm.segments = ['car_owner', 'pt_captive'] 

loaded_links: 100%|██████████| 40/40 [00:00<00:00, 81.01it/s]    


In [38]:
sm = sm.change_epsg(4326,'degree')

Reprojecting model from epsg 32755 to epsg 4326: 100%|██████████| 40/40 [00:01<00:00, 36.54it/s]


# emission / attraction

In [39]:
df = sm.zones_poly[['geometry']].copy()

In [40]:
emission = sm.volumes.groupby('origin').agg(sum)
emission.columns =  'emission_' + emission.columns

attraction = sm.volumes.groupby('destination').agg(sum)
attraction.columns =  'attraction_' + attraction.columns

In [41]:
df = df.merge(emission,left_index=True,right_index=True)
df = df.merge(attraction,left_index=True,right_index=True)

In [42]:
df.to_file(os.path.join(output_folder,'volumes.geojson'), driver='GeoJSON')

# serpents de charge

In [43]:
gpd.GeoDataFrame(sm.nodes,crs=sm.epsg).to_crs(4326).to_file(os.path.join(output_folder,'loaded_nodes.geojson'), driver='GeoJSON')

In [44]:
gpd.GeoDataFrame(sm.links.drop('road_link_list', axis=1, errors='ignore'),crs=sm.epsg).to_file(os.path.join(output_folder,'loaded_links.geojson'), driver='GeoJSON')

# boardings

In [45]:
pd.DataFrame(sm.links.groupby(['route_type'])['boardings'].sum()).to_csv(os.path.join(output_folder,'boardings_route_type.csv'))

In [46]:
pd.DataFrame(sm.links.groupby(['agency_id'])['boardings'].sum()).to_csv(os.path.join(output_folder,'boardings_agency.csv'))

In [47]:
pd.DataFrame(sm.links.groupby('route_id')['boardings'].sum()).to_csv(os.path.join(output_folder,'boardings_route_id.csv'))

In [48]:
pd.DataFrame(sm.links.loc[sm.links['agency_id'] == 'QUENEDI'].groupby('route_id')['boardings'].sum()).to_csv(os.path.join(output_folder,'boardings_quenedi.csv'))

# Path

In [49]:
od_file = scenario_folder + 'od/od.geojson'
od_file_provided = os.path.isfile(od_file)
if od_file_provided:
    od_test = gpd.read_file(od_file)
    from shapely.geometry import Point
    from syspy.spatial.spatial import nearest
    od_test['geometry_o'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][0]))
    od_test['geometry_d'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][1]))
    # find nearest node with KNN. nodes are now the origin and destination.
    od_test['geometry'] = od_test['geometry_o']
    neigh = nearest(od_test, sm.zones.to_crs(4326), n_neighbors=1).rename(columns={'ix_one': 'zone_index', 'ix_many': 'node_index'})
    zone_node_dict = neigh.set_index('zone_index')['node_index'].to_dict()
    od_test['origin'] = od_test.index.map(zone_node_dict.get)

    od_test['geometry'] = od_test['geometry_d']
    neigh = nearest(od_test, sm.zones.to_crs(4326), n_neighbors=1).rename(columns={'ix_one': 'zone_index', 'ix_many': 'node_index'})
    zone_node_dict = neigh.set_index('zone_index')['node_index'].to_dict()
    od_test['destination'] = od_test.index.map(zone_node_dict.get)

    od_test = od_test.drop(columns=['geometry_o','geometry_d','geometry'])
    od_test = od_test.drop(columns=['index'])
else:
    end_of_notebook

In [50]:
#only keep best path
sm.pt_los = sm.pt_los[sm.pt_los['pathfinder_session'] == 'best_path']

In [51]:
if 'speed' not in sm.links.columns:
    sm.links['speed'] = sm.links['length']/sm.links['time']*3.6

In [52]:
#PT links
od_links = od_test.merge(sm.pt_los[['origin','destination','gtime','link_path','pathfinder_session']],
                         on=['origin','destination'])
od_links = od_links.drop(columns = ['origin','destination'])
od_links = od_links.explode('link_path')
od_links = od_links.merge(sm.links[['route_color','geometry','time','speed']],left_on='link_path',right_index=True)
od_links = od_links.drop(columns='link_path')

In [53]:
#ntlegs
od_ntlegs = od_test.merge(sm.pt_los[['origin','destination','gtime','ntlegs','pathfinder_session']],
                          on=['origin','destination'])
od_ntlegs = od_ntlegs.drop(columns = ['origin','destination'])
od_ntlegs = od_ntlegs.explode('ntlegs')
ntlegs_dict = sm.zone_to_transit.reset_index().set_index(['a','b'])['index'].to_dict()
od_ntlegs['ntlegs'] = od_ntlegs['ntlegs'].apply(ntlegs_dict.get)
od_ntlegs = od_ntlegs.merge(sm.zone_to_transit[['geometry','time','speed']],left_on='ntlegs',right_index=True)
od_ntlegs = od_ntlegs.drop(columns='ntlegs')

In [54]:
sm.footpaths['speed'] = sm.footpaths['length']/sm.footpaths['time']*3.6

In [55]:
#footpath
od_footpaths = od_test.merge(sm.pt_los[['origin','destination','gtime','footpaths','pathfinder_session']], 
                             on=['origin','destination'])
od_footpaths = od_footpaths.drop(columns = ['origin','destination'])
od_footpaths = od_footpaths.explode('footpaths')
footpaths_dict = sm.footpaths.reset_index().set_index(['a','b'])['index'].to_dict()
od_footpaths['footpaths'] = od_footpaths['footpaths'].apply(footpaths_dict.get)
od_footpaths = od_footpaths.merge(sm.footpaths[['geometry','time','speed']],left_on='footpaths',right_index=True)
od_footpaths = od_footpaths.drop(columns='footpaths')

In [56]:
# raod links (walk on road)
od_road = od_test.merge(sm.pt_los[['origin','destination','gtime','footpaths','pathfinder_session']], 
                             on=['origin','destination'])
od_road = od_road.drop(columns = ['origin','destination'])
od_road = od_road.explode('footpaths')
sm.road_links['index'] = sm.road_links.index
road_dict = sm.road_links.set_index(['a','b'])['index'].to_dict()
od_road['footpaths'] = od_road['footpaths'].apply(road_dict.get)
od_road = od_road.merge(sm.road_links[['geometry','time','speed']],left_on='footpaths',right_index=True)
od_road = od_road.drop(columns='footpaths')

In [57]:
#road to transit
od_rt = od_test.merge(sm.pt_los[['origin','destination','gtime','footpaths','pathfinder_session']], 
                             on=['origin','destination'])
od_rt = od_rt.drop(columns = ['origin','destination'])
od_rt = od_rt.explode('footpaths')
road_dict = sm.road_to_transit.reset_index().set_index(['a','b'])['index'].to_dict()
od_rt['footpaths'] = od_rt['footpaths'].apply(road_dict.get)
od_rt = od_rt.merge(sm.road_to_transit[['geometry','time','speed']],left_on='footpaths',right_index=True)
od_rt = od_rt.drop(columns='footpaths')

In [58]:
od_route = pd.concat([od_links,od_footpaths,od_ntlegs,od_road,od_rt],axis=0)

In [59]:
od_route['route_color'] = od_route['route_color'].fillna('838383')
od_route['route_color'] = '#' + od_route['route_color']

In [60]:
od_route = od_route.rename(columns={'name':'od_name'})
od_route.reset_index(drop=True)
od_route.index.name='index'

In [61]:
od_route = gpd.GeoDataFrame(od_route,crs=4326)
od_route.to_file(output_folder+'od_route.geojson',driver='GeoJSON',engine=io_engine)

In [62]:
end_of_notebook

NameError: name 'end_of_notebook' is not defined